In [180]:
# import
import keras
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.utils import np_utils, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn import model_selection
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support
import h5py as h5py

In [181]:
BINARY = False
timestep = 1
epochs = 10
en_corpus_file = "corpus-en.txt"
ewo_corpus_file = "corpus-ewo.txt"
best_model_file = "best-model-conll.hdfs"
max_nb_of_phrases =  -1
duplication = 1
max_depth = 7
is_only_vocab = True

In [182]:
def getTag(aString):
    tag = "O"
    if BINARY:
        if aString != "O":
            return "NE"
    else:
        tag = aString
    return tag
     

In [183]:
def load_corpus(file, max_nb_of_phrases):
    nb_of_phrases = 0
    dataset = {"word": [], "ne-tag": []}
    with open(file) as f:
        prev_line = None
        for cpt, line in enumerate(f):
            if cpt == 0:
                continue
            if nb_of_phrases == max_nb_of_phrases:
                break;

            l = line.strip()
            if len(l) == 0 and len(prev_line) != 0:
                nb_of_phrases += 1
                dataset["word"].append(line)
                dataset["ne-tag"].append(None)
            else:
                l = l.split("\t")
                dataset["word"].append(l[0])
                dataset["ne-tag"].append(ne_type(l[1]))
            prev_line = line.strip()
        
    return pd.DataFrame(dataset), nb_of_phrases+1

In [184]:
def corpus_fingerprint(aDataframe, nb_of_biphrases):
    # create distributionnal signature
    fingerprints = {}
    current_bi_phrase_index = 0
    for index, row in aDataframe.iterrows():
        if current_bi_phrase_index > nb_of_biphrases:
            break
            
        word = row['word']
        
        if word != "\n":
            if word not in fingerprints:
                fingerprints[word] = np.zeros(nb_of_biphrases, dtype=np.int8)
            fingerprints[word][current_bi_phrase_index] = 1
        else:
            current_bi_phrase_index += 1
    return pd.DataFrame(fingerprints)

In [185]:
def corpus2trainingdata(aDataframe, fingerprintsDataFrame):
    X = np.zeros((aDataframe.shape[0], fingerprintsDataFrame.shape[0]), dtype=np.int8)
    y = np.zeros(aDataframe.shape[0], dtype=np.int8)
    i = 0
    for row in aDataframe.iterrows():
        X[i] = fingerprintsDataFrame[row[1]['word']].values
        y[i] = tag2int[getTag(row[1]['ne-tag'])]
        i += 1
    return X, y

In [186]:
def train_test_split(X, y, test_size = 0.33):
    total = X.shape[0]
    train_length = round(total * (1 - test_size)) 
    return X[:train_length], X[train_length:], y[:train_length], y[train_length:]

In [187]:
  
def ne_type(aType):
    aType = aType.lower()
    if 'per' in aType:
        t =  'NE' if BINARY else 'PER' 
    elif 'loc' in aType:
        t =  'NE' if BINARY else 'LOC'
    elif 'org' in aType:
        t =  'NE' if BINARY else 'ORG'
    elif 'hour' in aType:
        t =  'NE' if BINARY else 'MISC'
    elif aType != 'o' and len(aType) > 0 :
        t =  'NE' if BINARY else 'MISC'
    else:
        t = 'O'
    return t

In [188]:
def compute_performance(y_true, y_pred, words=None, BINARY=False):
    if BINARY:
        p = precision_score(y_true, y_pred, pos_label=tag2int['NE'])
        r = recall_score(y_true, y_pred, pos_label=tag2int['NE'])
        f1 = f1_score(y_true, y_pred, pos_label=tag2int['NE'])
        acc = accuracy_score(y_true, y_pred)
    else:
        p = precision_score(y_pred, y_true, average='macro')
        r = recall_score(y_pred, y_true, average='macro')
        f1 = f1_score(y_pred, y_true, average='macro')
        acc = accuracy_score(y_pred, y_true)
    if words is None:
        model_output_vs = pd.DataFrame({'y_true': [int2tag[i] for i in y_true], 'y_pred': [int2tag[i] for i in y_pred]})
    else:
        model_output_vs = pd.DataFrame({'word': words, 'y_true': [int2tag[i] for i in y_true], 'y_pred': [int2tag[i] for i in y_pred]})

    return p, r, f1, acc, model_output_vs

In [189]:
def P_R_F1(y_pred, y_true, neg_class):
    same = y_pred[y_true==y_pred]
    tp = same[same != neg_class].size
    nb_of_pos_exple = y_true[y_true != neg_class].size
    nb_of_pos_pred = y_pred[y_pred != neg_class].size
    p = r = f1 = 0
    try:
        p = np.round(tp*100/nb_of_pos_pred, 2)
    except ZeroDivisionError:
        print("number of correct positive predictions is 0")
        
    try:
        r = np.round(tp*100/nb_of_pos_exple, 2)
    except ZeroDivisionError:
        print("number of position exple is 0")
        
    try:
        f1 = np.round(2*r*p/(r+p), 2)
    except ZeroDivisionError:
        print("Recall and precision are 0")

    return p, r, f1

In [190]:
def merge(depth, corpus, fingerprint):
    text = list(corpus[corpus.word != "\n"].word)
    prev_w, prev_tag = None, None
    X2, target2=[], []
    tokens = []
    for level in range(depth):
        nvocab = text[:]
        for idx, w in enumerate(nvocab):
            word = w.split(" ")[0]
            if idx == 0:
                prev_w = w
                prev_tag = corpus[corpus.word==word].iloc[0]['ne-tag']
                continue
            prev_word = prev_w.split(" ")[0]
            tag = corpus[corpus.word==word].iloc[0]['ne-tag']
            if fingerprint[word].equals(fingerprint[prev_word]):
                if tag == prev_tag and tag != "O":
                    X2.append(fingerprint[word])
                    target2.append(tag2int[tag])
                    new_word = " ".join([prev_w, w])
                    tokens.append(new_word)
                    v = text[:idx-1] + [new_word] + text[idx+1:]
                    text = v[:]
                elif tag != 'O' and prev_tag == 'O':
                    X2.append(fingerprint[word])
                    target2.append(tag2int[tag])
                    new_word = " ".join([prev_w, w])
                    tokens.append(new_word)
                    v = text[:idx-1] + [new_word] + text[idx+1:]
                    text = v[:]
                else:
                    pass
            prev_w = w
            prev_tag = tag
    X2, target2 = np.array(X2), np.array(target2)
    return X2, target2, tokens

In [191]:
def shuffle(X, y):
    indices = [i for i in  range(X.shape[0])]
    np.random.shuffle(indices)
    return X[indices], y[indices]

In [192]:
tt, yy = np.array([i for i in range(15)]), np.array([i for i in range(15)])
print(tt, yy, sep="\n")

idxxx = [i for i in  range(15)]
np.random.shuffle(idxxx)
print(idxxx)

print(tt[idxxx], yy[idxxx], sep="\n")

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[12, 7, 14, 11, 9, 5, 4, 1, 8, 10, 2, 3, 13, 6, 0]
[12  7 14 11  9  5  4  1  8 10  2  3 13  6  0]
[12  7 14 11  9  5  4  1  8 10  2  3 13  6  0]


In [193]:
en_corpus, en_nb_of_phrases = load_corpus(en_corpus_file, max_nb_of_phrases)

In [194]:
tagSet = en_corpus["ne-tag"].dropna().unique()
if BINARY:
    tagSet = ['NE', 'O']
tag2int = {j: i for i, j in enumerate(tagSet)}
int2tag = {i: j for i, j in enumerate(tagSet)}
print(tag2int)

{'LOC': 3, 'PER': 2, 'O': 0, 'MISC': 1, 'ORG': 4}


In [195]:
en_nb_of_phrases

210

In [196]:
en_corpus.describe()

,ne-tag,word
count,4753,4962
unique,5,913
top,O,","
freq,4362,343


In [197]:
en_corpus.head(10)

,ne-tag,word
0,O,The
1,O,Promise
2,O,of
3,O,the
4,MISC,Holy
5,MISC,Spirit
6,None,\n
7,O,In
8,O,the
9,O,first


In [198]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(en_corpus[en_corpus['ne-tag']==tag].shape[0] * 100 / en_corpus[en_corpus['ne-tag']!='\n'].shape[0], 2)))

O % = 87.91 %
MISC % = 2.12 %
PER % = 4.94 %
LOC % = 0.81 %
ORG % = 0.02 %


In [199]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(en_corpus[en_corpus['ne-tag']==tag].word.unique().shape[0] * 100 / en_corpus[en_corpus['ne-tag']!='\n'].word.unique().shape[0], 2)))

O % = 89.27 %
MISC % = 1.86 %
PER % = 8.87 %
LOC % = 1.97 %
ORG % = 0.11 %


In [200]:
en_corpus[en_corpus.word == "\n"].shape

(209, 2)

In [201]:
print("Nb of bi-phrases", en_nb_of_phrases)
en_fingerprints = corpus_fingerprint(en_corpus, en_nb_of_phrases)

Nb of bi-phrases 210


In [202]:
en_corpus[en_corpus.word != "\n"].shape

(4753, 2)

In [203]:
X2, target2, tokens = merge(max_depth, en_corpus, en_fingerprints)

if is_only_vocab:
    text = list(en_corpus[en_corpus.word != "\n"].word.unique())
else:
    text = list(en_corpus[en_corpus.word != "\n"].word)
en_vocab = pd.DataFrame({'text': text + tokens})
en_vocab.describe()

,text
count,961
unique,926
top,Most High
freq,17


In [204]:
if is_only_vocab:
    X = np.zeros((en_vocab.shape[0] * duplication, en_nb_of_phrases))
    target = np.zeros((en_vocab.shape[0] * duplication))
    p=0
    for i, row in en_vocab.iterrows():
        c = row.text
        for j in range(duplication):
            X[p] = en_fingerprints[c.split(" ")[0]]
            target[p] = tag2int[getTag(en_corpus[en_corpus.word == c.split(" ")[-1:][0]]['ne-tag'].iloc[0])]
            p+=1
    X, target = shuffle(X, target)
    print(X.shape, en_fingerprints.shape, target.shape)

(961, 210) (210, 912) (961,)


In [205]:
en_vocab[-20:]

,text
941,Most High
942,Ju d
943,Ju d Tamar
944,Most Most High
945,Most High High Most High
946,Most High
947,Lo r
948,Most High
949,Ju d
950,Most Most High Most High High Most High


In [206]:
if not is_only_vocab:
    X, target = corpus2trainingdata(en_corpus[en_corpus.word != "\n"], en_fingerprints)

In [207]:
print(X.shape, target.shape)
if len(X.shape) == len(X2.shape):
    X = np.concatenate((X, X2))
    target = np.concatenate((target, target2))
    X, target = shuffle(X, target)
print(X.shape, target.shape)

(961, 210) (961,)
(1010, 210) (1010,)


In [208]:
y = target.copy()
y[0:100]
if not BINARY:
    y = np_utils.to_categorical(y, len(tagSet))
y.shape

(1010, 5)

In [209]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.33)
X_train = X_train.reshape(X_train.shape[0], timestep, X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], timestep, X_val.shape[1])
print("X_train.shape =", X_train.shape)
print("y_train.shape =", y_train.shape)
print("X_val.shape =", X_val.shape)
print("y_val.shape =", y_val.shape)

tTarget = np.array([np.argmax(yy) for yy in y_train])
vTarget = np.array([np.argmax(yy) for yy in y_val])

for tag in tagSet:
    print("{0} % in training data = {1} %".format(tag, np.round(tTarget[tTarget==tag2int[tag]].size * 100 / tTarget.shape[0], 2)))
    print("{0} % in validation data = {1} %".format(tag, np.round(vTarget[vTarget==tag2int[tag]].size * 100 / vTarget.shape[0], 2)))

X_train.shape = (676, 1, 210)
y_train.shape = (676, 5)
X_val.shape = (334, 1, 210)
y_val.shape = (334, 5)
O % in training data = 81.8 %
O % in validation data = 75.75 %
MISC % in training data = 6.95 %
MISC % in validation data = 7.78 %
PER % in training data = 7.99 %
PER % in validation data = 12.28 %
LOC % in training data = 3.11 %
LOC % in validation data = 4.19 %
ORG % in training data = 0.15 %
ORG % in validation data = 0.0 %


In [210]:
def create_model(input_dim, output_dim):
    model = Sequential()
    model.add(SimpleRNN(640, input_shape=(None, input_dim), activation='sigmoid'))
    model.add(Dense(160, activation='sigmoid'))
    if BINARY:
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])
    else:
        model.add(Dense(output_dim, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
    return model

In [211]:
# model = create_model(X_train.shape[2], len(tagSet))
# model.summary()

In [212]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=epochs):
    early_stop = EarlyStopping(patience=2, verbose=2) # stop learning if the error is the same between two consecutive epochs
    best_model_cp = ModelCheckpoint(best_model_file, save_best_only=True, verbose=2) # saved best model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, verbose=0, callbacks=[best_model_cp, early_stop])
    best_model = keras.models.load_model(best_model_file) #loading the best model
    return best_model

In [213]:
def model_performance(model, X, y, binary=BINARY):
    if BINARY:
        y_pred = np.round(model.predict(X))
        y_true = y
    else:
        predictions = model.predict(X)
        y_pred = np.array([np.argmax(p) for p in predictions])
        y_true = np.array([np.argmax(t) for t in y ])
    return P_R_F1(y_pred, y_true, tag2int['O']) #precision, recall, f1-score

In [214]:
ewo_corpus, ewo_nb_of_phrases = load_corpus(ewo_corpus_file, max_nb_of_phrases)

In [215]:
ewo_nb_of_phrases

210

In [216]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(ewo_corpus[ewo_corpus['ne-tag']==tag].shape[0] * 100 / ewo_corpus[ewo_corpus['ne-tag']!='\n'].shape[0], 2)))

O % = 86.37 %
MISC % = 2.18 %
PER % = 5.76 %
LOC % = 0.89 %
ORG % = 0.05 %


In [217]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(ewo_corpus[ewo_corpus['ne-tag']==tag].word.unique().shape[0] * 100 / ewo_corpus[ewo_corpus['ne-tag']!='\n'].word.unique().shape[0], 2)))

O % = 90.0 %
MISC % = 1.17 %
PER % = 8.25 %
LOC % = 1.84 %
ORG % = 0.19 %


In [218]:
ewo_corpus.describe()

,ne-tag,word
count,4185,4394
unique,5,1030
top,O,","
freq,3795,413


In [219]:
ewo_corpus.head()

,ne-tag,word
0,MISC,Mfufub
1,MISC,Nsisim
2,O,ayi
3,O,sò
4,None,\n


In [220]:
ewo_fingerprints = corpus_fingerprint(ewo_corpus, en_nb_of_phrases)

In [221]:
X2, target2, tokens = merge(max_depth, ewo_corpus, ewo_fingerprints)
if is_only_vocab:
    text = list(ewo_corpus[ewo_corpus.word != "\n"].word.unique())
else:
    text = list(ewo_corpus[ewo_corpus.word != "\n"].word)
ewo_vocab = pd.DataFrame({"text":text + tokens})

In [222]:
if is_only_vocab:
    X_ewo = np.zeros((ewo_vocab.shape[0] * duplication, en_nb_of_phrases))
    ewo_target = np.zeros((ewo_vocab.shape[0] * duplication))
    p=0
    for i, row in ewo_vocab.iterrows():
        c = row.text
        for j in range(duplication):
            X_ewo[p] = ewo_fingerprints[c.split(" ")[0]]
            ewo_target[p] = tag2int[getTag(ewo_corpus[ewo_corpus.word == c.split(" ")[-1:][0]]['ne-tag'].iloc[0])]
            p+=1
    X_ewo, ewo_target = shuffle(X_ewo, ewo_target)

In [223]:
ewo_vocab[-20:]

,text
1126,Mfufub Nsisim
1127,Mfufub Nsisim
1128,Mfufub Nsisim
1129,Mfufub Nsisim
1130,Mfufub Nsisim
1131,nkol nkol bile-bi-mbòn
1132,nkol bile-bi-mbòn bile-bi-mbòn nkol bile-bi-mbòn
1133,Mfufub Nsisim
1134,Afub mëki
1135,mëwola ebu


In [224]:
if not is_only_vocab:
    X_ewo, ewo_target = corpus2trainingdata(ewo_corpus[ewo_corpus.word != "\n"], ewo_fingerprints)

In [225]:
print(X_ewo.shape, ewo_target.shape)
if len(X_ewo.shape) == len(X2.shape):
    X_ewo = np.concatenate((X_ewo, X2))
    ewo_target = np.concatenate((ewo_target, target2))
    X_ewo, ewo_target = shuffle(X_ewo, ewo_target)
print(X_ewo.shape, ewo_target.shape)

(1146, 210) (1146,)
(1263, 210) (1263,)


In [226]:
y_ewo = ewo_target.copy()
print(y_ewo.shape, len(ewo_vocab))

(1263,) 1146


In [227]:
X_ewo.shape

(1263, 210)

In [228]:
y_ewo = ewo_target.copy()
y_ewo[:20]
if not BINARY:
    y_ewo = np_utils.to_categorical(y_ewo)

In [229]:
X_ewo = X_ewo.reshape((X_ewo.shape[0], timestep, en_nb_of_phrases))

In [230]:
def algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, repeat=100, epochs=epochs, model=None):
    test_precision, train_precision, ewo_precision = [], [], []
    test_recall, train_recall, ewo_recall = [], [], []
    test_fscore, train_fscore, ewo_fscore = [], [], []
    reset_model = model is None
    for i in range(repeat):
        print("AlgoEval Iter -", i+1)
        if reset_model :
            model = create_model(X_train.shape[2], len(tagSet))
        m = train_model(model, X_train, y_train, X_val, y_val, epochs=epochs)
        p_train, r_train, f1_train = model_performance(m, X_train, y_train)
        p_val, r_val, f1_val = model_performance(m, X_val, y_val)
        p_ewo, r_ewo, f1_ewo = model_performance(m, X_ewo, y_ewo)
        
        test_precision.append(p_val)
        train_precision.append(p_train)
        ewo_precision.append(p_ewo)
        
        test_recall.append(r_val)
        train_recall.append(r_train)
        ewo_recall.append(r_ewo)
        
        test_fscore.append(f1_val)
        train_fscore.append(f1_train)
        ewo_fscore.append(f1_ewo)
    return pd.DataFrame({
        'P_test': test_precision, 
        'P_train': train_precision, 
        'P_ewo': ewo_precision, 'R_test': test_recall, 'R_train': train_recall, 
        'R_ewo': ewo_recall, 'F1-test': test_fscore, 'F1-train': train_fscore, 'F1-ewo': ewo_fscore})

In [231]:
# resultEval = algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, repeat=10)

In [232]:
# resultEval

In [233]:
# resultEval.mean()

In [234]:
# resultEval.std()

In [235]:
def algoCrossVal(model, X, y, X_ewo, y_ewo, k = 4, repeat=1): 
    block_size = int(X.shape[0] / 4)   
    output = None
    for it in range(repeat):
        print("AlgoCrossValIter -", it+1)
        model = create_model(X.shape[1], len(tagSet))
        results = None
        for i in range(k):
            X_val, y_val = X[i*block_size:i*block_size+block_size], y[i*block_size:i*block_size+block_size]
            X_train = np.concatenate((X[0:i*block_size], X[i*block_size+block_size:]))
            y_train = np.concatenate((y[0:i*block_size], y[i*block_size+block_size:]))

            X_train = X_train.reshape(X_train.shape[0], timestep, X_train.shape[1])
            X_val = X_val.reshape(X_val.shape[0], timestep, X_val.shape[1])

            result = algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, repeat=1, model=model)
            if results is None:
                results = result.copy()
            else:
                results = pd.concat([results, result])
        
        if output is None:
            output = results.mean(axis=0).to_frame()
        else:
            output = pd.concat([output, results.mean(axis=0).to_frame()], axis=1)

    return output

In [236]:
m = create_model(X.shape[1], len(tagSet))
resultCrossVal = algoCrossVal(m, X, y, X_ewo, y_ewo, k=4, repeat=10)

AlgoCrossValIter - 1
AlgoEval Iter - 1

Epoch 00001: val_loss improved from inf to 0.73549, saving model to best-model-conll.hdfs

Epoch 00002: val_loss improved from 0.73549 to 0.71005, saving model to best-model-conll.hdfs

Epoch 00003: val_loss improved from 0.71005 to 0.65290, saving model to best-model-conll.hdfs

Epoch 00004: val_loss improved from 0.65290 to 0.61671, saving model to best-model-conll.hdfs

Epoch 00005: val_loss did not improve

Epoch 00006: val_loss improved from 0.61671 to 0.60324, saving model to best-model-conll.hdfs

Epoch 00007: val_loss improved from 0.60324 to 0.58336, saving model to best-model-conll.hdfs

Epoch 00008: val_loss improved from 0.58336 to 0.56300, saving model to best-model-conll.hdfs

Epoch 00009: val_loss improved from 0.56300 to 0.49579, saving model to best-model-conll.hdfs

Epoch 00010: val_loss improved from 0.49579 to 0.48358, saving model to best-model-conll.hdfs
AlgoEval Iter - 1

Epoch 00001: val_loss improved from inf to 0.38084, 


Epoch 00002: val_loss improved from 0.42850 to 0.40831, saving model to best-model-conll.hdfs

Epoch 00003: val_loss improved from 0.40831 to 0.37643, saving model to best-model-conll.hdfs

Epoch 00004: val_loss did not improve

Epoch 00005: val_loss improved from 0.37643 to 0.35946, saving model to best-model-conll.hdfs

Epoch 00006: val_loss improved from 0.35946 to 0.34180, saving model to best-model-conll.hdfs

Epoch 00007: val_loss improved from 0.34180 to 0.34022, saving model to best-model-conll.hdfs

Epoch 00008: val_loss did not improve

Epoch 00009: val_loss did not improve
Epoch 00009: early stopping
AlgoEval Iter - 1

Epoch 00001: val_loss improved from inf to 0.26836, saving model to best-model-conll.hdfs

Epoch 00002: val_loss improved from 0.26836 to 0.25550, saving model to best-model-conll.hdfs

Epoch 00003: val_loss did not improve

Epoch 00004: val_loss improved from 0.25550 to 0.25525, saving model to best-model-conll.hdfs

Epoch 00005: val_loss did not improve

Ep


Epoch 00001: val_loss improved from inf to 0.37657, saving model to best-model-conll.hdfs

Epoch 00002: val_loss improved from 0.37657 to 0.37087, saving model to best-model-conll.hdfs

Epoch 00003: val_loss improved from 0.37087 to 0.31149, saving model to best-model-conll.hdfs

Epoch 00004: val_loss did not improve

Epoch 00005: val_loss did not improve
Epoch 00005: early stopping
AlgoEval Iter - 1

Epoch 00001: val_loss improved from inf to 0.25513, saving model to best-model-conll.hdfs

Epoch 00002: val_loss improved from 0.25513 to 0.25288, saving model to best-model-conll.hdfs

Epoch 00003: val_loss improved from 0.25288 to 0.24345, saving model to best-model-conll.hdfs

Epoch 00004: val_loss improved from 0.24345 to 0.24154, saving model to best-model-conll.hdfs

Epoch 00005: val_loss did not improve

Epoch 00006: val_loss improved from 0.24154 to 0.24089, saving model to best-model-conll.hdfs

Epoch 00007: val_loss did not improve

Epoch 00008: val_loss did not improve
Epoch 0

In [237]:
resultCrossVal

,0,0,0,0,0,0,0,0,0,0
F1-ewo,68.5925,70.560000,70.653333,69.7200,72.553333,53.4375,52.6300,71.066667,72.020000,71.950000
F1-test,56.8500,64.286667,69.263333,64.5125,66.577500,58.7000,59.6575,70.233333,66.553333,68.496667
F1-train,55.6850,57.863333,65.506667,61.4050,68.160000,57.6300,56.5725,68.966667,68.786667,72.403333
P_ewo,94.0625,71.147500,67.765000,91.9800,67.812500,82.4000,75.8550,69.872500,71.305000,67.805000
P_test,93.6500,73.075000,70.105000,94.3725,93.797500,96.1550,93.9550,72.337500,71.087500,68.010000
P_train,93.6175,70.492500,68.000000,90.1300,92.472500,94.6275,94.2150,69.082500,70.557500,67.350000
R_ewo,54.1650,42.325000,43.640000,56.3575,45.540000,43.1300,42.3250,43.202500,43.640000,44.957500
R_test,42.7875,37.322500,42.422500,51.4325,55.307500,45.1600,45.4050,42.280000,39.220000,42.280000
R_train,41.3300,33.720000,41.017500,49.2625,57.517500,44.5150,42.6250,42.257500,41.775000,46.500000


In [238]:
resultCrossVal.mean(axis=1)

F1-ewo      67.318333
F1-test     64.513083
F1-train    63.297917
P_ewo       76.000500
P_test      82.654500
P_train     81.054500
R_ewo       45.928250
R_test      44.361750
R_train     44.052000
dtype: float64

In [239]:
resultCrossVal.std(axis=1)

F1-ewo       7.619128
F1-test      4.663359
F1-train     6.165486
P_ewo       10.042907
P_test      12.455690
P_train     12.698619
R_ewo        5.048711
R_test       5.399851
R_train      6.222585
dtype: float64